<a href="https://colab.research.google.com/github/joshuadollison/smallbizpulse/blob/jd-model/notebooks/model_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# SETUP: Mount Drive, Install Dependencies, Configure Styling
# ============================================================

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install VADER for sentiment analysis
!pip install vaderSentiment -q

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from collections import Counter
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ── Consistent Plot Styling ──────────────────────────────────
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 12,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
})

# SmallBizPulse color palette
COLORS = {
    'primary': '#2563EB',
    'secondary': '#F59E0B',
    'open': '#10B981',
    'closed': '#EF4444',
    'accent1': '#8B5CF6',
    'accent2': '#EC4899',
    'neutral': '#6B7280',
    'bg': '#F9FAFB',
}
PALETTE_OC = [COLORS['open'], COLORS['closed']]

print("Setup complete — libraries loaded, styling configured.")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.2 MB/s eta 0:00:00
Setup complete — libraries loaded, styling configured.


In [2]:
# ============================================================
# DATA LOADING
# ============================================================
# >>> UPDATE THIS PATH to match your Google Drive folder <<<
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIS509/yelp_dataset_new/'

def load_json(filename):
    filepath = DATA_PATH + filename
    with open(filepath, 'r') as f:
        first_char = f.read(1)
        f.seek(0)
        if first_char == '[':
            return pd.DataFrame(json.load(f))
        else:
            return pd.read_json(f, lines=True)

print("Loading datasets...")
business_df = load_json('yelp_academic_dataset_business.json')
print(f"  Business: {len(business_df):,} records")

review_df = load_json('yelp_academic_dataset_review.json')
print(f"  Review:   {len(review_df):,} records")

tip_df = load_json('yelp_academic_dataset_tip.json')
print(f"  Tip:      {len(tip_df):,} records")

checkin_df = load_json('yelp_academic_dataset_checkin.json')
print(f"  Checkin:  {len(checkin_df):,} records")

user_df = load_json('yelp_academic_dataset_user.json')
print(f"  User:     {len(user_df):,} records")

print("\nAll datasets loaded successfully.")

Loading datasets...
  Business: 9,973 records
  Review:   100,000 records
  Tip:      264,693 records
  Checkin:  9,337 records
  User:     79,345 records

All datasets loaded successfully.


In [ ]:
# ============================================================
# DATA SOURCES & FILTERING CRITERIA
# ============================================================

# Step 1: Filter for restaurants
# A business is classified as a "restaurant" if its Yelp categories
# contain the word "Restaurants" (case-insensitive).
restaurant_df = business_df[
    business_df['categories'].str.contains('Restaurants', case=False, na=False)
].copy()

# Step 2: Get restaurant business IDs
restaurant_ids = set(restaurant_df['business_id'])

# Step 3: Filter reviews to restaurant-only
rest_review_df = review_df[review_df['business_id'].isin(restaurant_ids)].copy()
rest_review_df['date'] = pd.to_datetime(rest_review_df['date'])
rest_review_df['year'] = rest_review_df['date'].dt.year

# Step 4: Filter tips to restaurant-only
rest_tip_df = tip_df[tip_df['business_id'].isin(restaurant_ids)].copy()

# Step 5: Filter checkins to restaurant-only
rest_checkin_df = checkin_df[checkin_df['business_id'].isin(restaurant_ids)].copy()

# Step 6: Merge business status onto reviews
status_map = restaurant_df.set_index('business_id')['is_open'].to_dict()
rest_review_df['is_open'] = rest_review_df['business_id'].map(status_map)
rest_review_df['status'] = rest_review_df['is_open'].map({1: 'Open', 0: 'Closed'})

# ── Print Summary ────────────────────────────────────────────
print("=" * 65)
print("DATA SOURCES & FILTERING SUMMARY")
print("=" * 65)

print("\nPRIMARY DATA SOURCE: Yelp Academic Dataset")
print("-" * 45)

print("\nFull Dataset:")
print(f"  Businesses:  {len(business_df):>8,}")
print(f"  Reviews:     {len(review_df):>8,}")
print(f"  Tips:        {len(tip_df):>8,}")
print(f"  Check-ins:   {len(checkin_df):>8,}")
print(f"  Users:       {len(user_df):>8,}")

print("\nFiltered to Restaurants (categories contain 'Restaurants'):")
print(f"  Restaurants:        {len(restaurant_df):>8,}")
print(f"  Restaurant Reviews: {len(rest_review_df):>8,}")
print(f"  Restaurant Tips:    {len(rest_tip_df):>8,}")
print(f"  Restaurant Checkins:{len(rest_checkin_df):>8,}")

n_open = (restaurant_df['is_open'] == 1).sum()
n_closed = (restaurant_df['is_open'] == 0).sum()
print("\nRestaurant Status:")
print(f"  Open:   {n_open:>5,}  ({n_open / len(restaurant_df) * 100:.1f}%)")
print(f"  Closed: {n_closed:>5,}  ({n_closed / len(restaurant_df) * 100:.1f}%)")

print(f"\nDate Range: {rest_review_df['date'].min().strftime('%Y-%m-%d')} to "
      f"{rest_review_df['date'].max().strftime('%Y-%m-%d')}")

print("\nFILTERING CRITERIA APPLIED:")
print("  1. Category filter: categories.str.contains('Restaurants')")
print("  2. Reviews, tips, and check-ins filtered by restaurant business_id")
print("  3. No minimum review count threshold (preserving data-sparse")
print("     businesses is important for studying closure patterns)")
print("  4. No date range restriction (full temporal span needed for time-series)")